### Retriever

In [11]:
from langchain_community.vectorstores import FAISS as VectorStore
from langchain_community.embeddings import GPT4AllEmbeddings

embeddings = GPT4AllEmbeddings()
store = VectorStore.load_local("../../retrieve/vector_store", embeddings, allow_dangerous_deserialization=True)
retriever = store.as_retriever(search_kwargs={"k": 4})

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Model

In [12]:
import getpass

my_account_id = getpass.getpass("Enter your Cloudflare account ID:\n\n")
my_api_token = getpass.getpass("Enter your Cloudflare API token:\n\n")
llm_model = "@hf/mistralai/mistral-7b-instruct-v0.2"

In [13]:
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

llm = CloudflareWorkersAI(account_id=my_account_id, api_token=my_api_token, model=llm_model)

### Chains
https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs/

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

chain = load_qa_chain(llm=llm, chain_type="stuff", memory=memory, prompt=prompt)

In [15]:
%time
query = "How install the chatbots"
docs = retriever.invoke(query)

chain.invoke({"input_documents": docs, "human_input": query}) #, return_only_outputs=True)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 10 µs


{'input_documents': [Document(page_content="The chatbots are in their respective folders with an application launcher `autorun.sh` to install each of them without specific knowledge.  \nOn Mac, open the terminal and type:\n```shell\ncd\n```\nDrag the **`folder`** containing the file `autorun.sh`, then press the Enter key (↩︎).  \n_If you have done it correctly, the **`~`** between your machine's name (`name@MacBook-Pro-of-Name`) and the **`%`** sign should display the name of the `folder` instead._  \nExecute the following line of code by pressing the Enter key (↩︎):\n```shell\nsh autorun.sh\n```\nWait a moment, the model should open in your default web browser.", metadata={'Header 1': 'Models', 'Header 2': 'Installation', 'Header 3': 'Streamlit & FastAPI'}),
  Document(page_content='The subfolders listed below contain key steps in our research for creating a functional, convenient, and maintainable chatbot.', metadata={'Header 1': 'Onboarding Bot Model', 'Header 2': 'Structure'}),
  D

### Some interesting documents

 https://python.langchain.com/docs/get_started/quickstart/#conversation-retrieval-chain \
 https://python.langchain.com/docs/use_cases/chatbots/quickstart/ \
 https://python.langchain.com/docs/use_cases/chatbots/retrieval/ \
 https://python.langchain.com/v0.1/docs/integrations/llms/cloudflare_workersai/ 
 https://api.python.langchain.com/en/latest/llms/langchain_community.llms.cloudflare_workersai.CloudflareWorkersAI.html

#### default in CloudFlare documentation 
```python
llm_model = "@cf/meta/llama-2-7b-chat-int8" 
```
#### default in HuggingFace deploy buton 
```python
llm_model = "@hf/mistralai/mistral-7b-instruct-v0.2" 
```
